In [ ]:
import cv2
import time
from pinkylib import Pinky, Camera
from pinkylib.yolo import Yolo

In [ ]:
APPLE = 0 # 사과 갯수를 저장할 변수
target_id_list = [8, 7, 11, 9] #아루코마커 주행 순서 id 리스트 [교차로진입, 사과나무, 교차로이탈, 종료]
target_id_index = 0 #target_id_list의 
apple_ids = [7] # 사과나무가 있는 아루코마커 id 인덱스 (목표 좌표 도착시 1씩 증가)
turn_right_ids = [8, 11] # 설정된 좌표에 도착할 시 오른쪽 회전 동작하는 아루코마커 id 리스트
end_id = 9 # 주행 종료 id 설정
pose = {             # 마커 id의 도착 위치 설정 { [x,z]좌표 (x는 마커와 로봇의 좌우 좌표, z는 거리)}
    8 : [0.33, 20],
    7 : [2.6, 14],
    11 : [-0.98, 16],
    9 : [1.9, 9]
}

In [ ]:
pinky = Pinky()
pinky.start_motor()
pinky.enable_motor()

In [ ]:
cam = Camera()
cam.start()
cam.set_calibration()

In [ ]:
yolo = Yolo()
yolo.set_model("./best.pt")

In [ ]:

def find_aruco():
    pinky.turn_right(30, 30)
    time.sleep(0.2)
    pinky.stop()
    time.sleep(1)

In [ ]:
def turn_right():
    pinky.turn_right(30, 30)
    time.sleep(0.4)
    pinky.stop()

In [ ]:
def turn_180():
    pinky.turn_right(30, 30)
    time.sleep(1)
    pinky.stop()

In [ ]:
def find_apple(frame):
    global APPLE
    pinky.stop()
    clss, frame = yolo.detect_yolo(frame)
    for i in clss:
        if i == 0:
            APPLE += 1

    return frame

In [ ]:
# 시작 부저
pinky.buzzer_start()
pinky.buzzer(2)

while True:
    # 0으로 속도값 초기화
    L, R = 0, 0
    
    frame = cam.get_frame()
    
    target_id = target_id_list[target_id_index] # 목표 아루코마커 id 설정
    target_x, target_z = pose[target_id] # 목표 좌표 설정
    print(f"target id is {target_id}")

    aruco_pose, frame = cam.detect_aruco_target(frame, target_id, marker_size=0.036) #인식된 마커의 좌표
    print(aruco_pose)
    
    # 인식된 마커가 있을 경우
    if aruco_pose is not None:
        # 인식된 마커와 로봇과의 x, z 좌표
        aruco_x, aruco_z = aruco_pose[0], aruco_pose[2]
        
        # 로봇이 목표 z좌표안에 도착하고 목표 x좌표의 좌우 0.3 안에 위치하면 실행
        # x좌표를 한 좌표를 목표로 설정해서 주행하면 좌우로 계속 흔들리면서 주행하므로 좌우 0.3 가중치 추가해서 흔들리면서 주행 방지
        if aruco_z < target_z and aruco_x < target_x + 0.3 and aruco_x > target_x - 0.3 : 
            print("stop")
            
            # target_id가 turn_right_ids에 있으면 실행
            # 오른쪽으로 90도 회전
            if target_id in turn_right_ids: 
                print("trun right")
                turn_right()
            
            # target_id가 apple_ids에 있으면 실행
            # 사과 인식
            elif target_id  in apple_ids:
                print("detect apple")
                frame = find_apple(frame)
                print(f"현재 인식한 사과 {APPLE} 개")
                print("turn 180")
                turn_180()

            # target_id가 end_id 이면 실행:
            # 주행 종료
            elif target_id == end_id:
                print("fin")
                pinky.stop()
                break
            
            # 도착시 실행되는 동작 수행 후 인덱스 증가(다음 타깃 id 설정)
            target_id_index += 1
        
        else:
            # 로봇과 아루코마커의 거리가 목표 거리보다 크면 직진 
            if target_z < aruco_z:
                L, R = 25, 25

            # 로봇 기준으로 아루코마커가 목표 x좌표 보다 오른쪽에 있으면 오른쪽으로 주행
            if aruco_x > target_x + 0.3:
                L = 30
            
            # 로봇 기준으로 아루코마커가 목표 x좌표 보다 왼쪽에 있으면 왼쪽으로 주행
            elif aruco_x < target_x - 0.3:
                R = 30
    
    # 인식된 마커가 없을 때 실행
    else:
        print("find aruco")
        find_aruco()
    
    # 위 조건문들 바탕으로 나온 속도값으로 주행
    pinky.move(L, R)

    cam.display_jupyter(frame)
    time.sleep(0.01)

# 종료 부저
pinky.buzzer(2)

In [ ]:
cam.close()